# Comment utiliser le notebook.

Vous trouverez des cellules contenant du texte et des blocs interactifs. Pour exécuter les fonctions interactives, cliquez sur la cellule, puis cliquez sur le button Run il se trouve sur la parti supérieure du notebook). Il est important de suivre l'ordre des blocs et exécuter tous les blocs précédents pour utiliser les blocs interactifs. 

In [ ]:
from IPython.display import Image

import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead, BoxAnnotation, Legend
from bokeh.layouts import gridplot
output_notebook()

# L'agent Logan : exercice de cinématique

Lancé à la poursuite d'un criminel, l'agent Logan du FBI doit traverser une rivière d'une largeur de 1600 m qui coule à 0.80 m$\cdot$s$^{-1}$ en un minimum de temps et se rendre directement en face de son point de départ. Sachant qu'il peut ramer à 1.50 m$\cdot$s$^{-1}$ et courir à 3.00 m$\cdot$s$^{-1}$, décrivez la route qu'il devrait suivre (en bateau et à pied le long de la rive) pour traverser ce cours d'eau le plus rapidement possible.
Déterminez le temps minimal requis pour cette traversée.


## On commence par un schéma...

In [ ]:
Image("Logan.png")

## On analyse le problème et on affine  la marche à suivre

Logan doit aller de A à B le plus vite possible (pas forcément en ligne droite), 
sa vitesse par rapport à la rivière est $\vec v'$, la vitesse du courant $\vec v_c$, donc sa vitesse par rapport à la rive est $\vec v = \vec  v' + \vec v_c$. 

Il choisit l'angle $\beta$. Si il ne choisit pas un beta tel que $\alpha$ vaille 90°, il ne va pas débarquer en B, et devra courir un peu sur la rive. Mais comme il court plus vite qu'il ne rame, cela peut être une bonne idée si il choisit un beta qui va lui permettre de gagner du temps sur la traversée. 

Le but est de choisir l'angle qui lui donne le meilleur temps. C'est un problème d'optimisation. On exprime le temps total en fonction du paramètre libre ($\beta$) et on cherche la valeur de $\beta$ qui donne le minimum.

### Fonctions et paramètres

In [ ]:
# Définition des paramètres initiaux
l = 1600 #largeur de la rivière (m)
v_courant = 0.8 # vitesse du courant (m/s)
v_ramer = 1.5 # vitesse à la rame (m/s)
v_course = 3.0 # vitesse à la course (m/s)

beta = 90 # angle de départ

## Calcul analytique 1

Le temps de traversée à la rame peut être calculé dans le référentiel de l'eau de la rivière.

$$t_r=\frac{l}{v_\text{ramer} \sin \beta}$$

Le temps de course nécessite de calculer d'abord la distance à parcourir sur la rive

vitesse du courant: $\vec v_c$

vitesse par rapport au courant: $\vec v'$ avec $|\vec v'|=v'$ connu ( appelé $v_{ramer}$ dans le programme)

vitesse par rapport à la rive: $\vec v=\vec v'+\vec v_c$

Composantes des vitesses:

\begin{align*}
\vec v_c&=\begin{pmatrix}v_c\\0 \end{pmatrix} & \vec v&=\begin{pmatrix}v_x=v\cos\alpha\\v_y=v\sin\alpha \end{pmatrix} & \vec v'&=\begin{pmatrix}v'\cos\beta\\v'\sin\beta \end{pmatrix}
\end{align*}
Il vient:

$$\vec v=\begin{pmatrix}v_c+v'\cos\beta\\v'\sin\beta \end{pmatrix}=\begin{pmatrix}v\cos\alpha\\v\sin\alpha \end{pmatrix}$$

Ainsi: $$\tan\alpha=\frac{v\sin\alpha}{v\cos\alpha}=\frac{v'\sin\beta}{v_c+v'\cos\beta}=\frac ld$$

Donc:
$$\boxed{d=l\frac{v_c+v'\cos\beta}{v'\sin\beta}}$$

Le temps de course sur la rive est donc donné par 

$$t_c=l\frac{v_c+v'\cos\beta}{v'\sin\beta}\frac{1}{v_{course}}=l\frac{v_\text{courant}+v_\text{ramer}\cos\beta}{v_\text{ramer}\sin\beta}\frac{1}{v_\text{course}} $$

Le temps total est $t_c+t_r$

### Vitesses

In [ ]:
# vecteur vitesse du rameur par rapport au courant (v') en fonction de l'angle beta (variable) et 
# de la norme de la vitesse à la rame (v_ramer)

v_ramer_ = lambda beta, v_ramer: [np.linspace(0, 200*np.cos(beta)*v_ramer, 10), \
                                  np.linspace(0, 200*np.sin(beta)*v_ramer, 10)]

# vecteur vitesse par raport au courant
v_ramer_x, v_ramer_y = v_ramer_(beta, v_ramer)

# vecteur vitesse courant
v_courant_x = v_ramer_x[-1] + np.linspace(0, 300*v_ramer, 10)
v_courant_y = [v_ramer_y[-1]]*10

# vitesse (vectorielle) initiale en x et y 
v_init_x = v_ramer*np.cos(beta)
v_init_y = v_ramer*np.sin(beta)

# angle alpha
alpha = np.arctan2(v_courant_y[-1],v_courant_x[-1])
print("Angle entre la trajectoire dans l'eau et la rive: {:0.4f} [rad]".format(alpha))

### Temps

In [ ]:
# temps de traversée à la rame 
t_ramer = lambda v_ramer, beta: l/(v_ramer*np.sin(beta))

t_r = t_ramer(v_ramer, beta)

# temps de course
t_course = lambda v_courant, v_ramer, beta: l*(v_courant + v_ramer*np.cos(beta))/(v_ramer*np.sin(beta) * v_course)

# temps total 
t_total = t_course(v_courant, v_ramer, beta) + t_ramer(v_ramer, beta)

print('Temps de traversée à la rame {:0.2f}[s]'.format(t_r))
print('Temps de course sur la rive {:0.2f}[s]'.format(t_course(v_courant, v_ramer, beta)))
print('Temps total {:0.2f}[s]'.format(t_course(v_courant, v_ramer, beta)))

### Point d'arrivée

In [ ]:
# distance de course (donne le point d'arrivée)
d = (v_courant + v_ramer*np.cos(beta))*t_r
print('Distance de course {:0.2f}[m]'.format(d))

# Représentation graphique 

Dans cette partie du Notebook, on définira les différents vecteurs qu'on veut représenter ainsi comme la figure globale. 

### Définition de la Figure

In [ ]:
# figure
p = figure(title="Logan", plot_height=500, plot_width=980, y_range=(-100,1800), x_range=(-2000,3000), 
           background_fill_color='#ffffff', toolbar_location="below")

# rives
r_down = p.line(np.linspace(-5000, 5000, 100), [0]*100, color="#5A2806", alpha=0.6, line_width=5)
r_down = p.line(np.linspace(-5000, 5000, 100), [l]*100, color="#5A2806", alpha=0.6, line_width=5)

# fons de l'image
mid_box = BoxAnnotation(bottom=0, top=l, fill_alpha=0.1, fill_color='blue')
low_box = BoxAnnotation(bottom=-100, top=0, fill_alpha=0.1, fill_color='green')
top_box = BoxAnnotation(bottom=l, top=1800, fill_alpha=0.1, fill_color='green')
p.add_layout(mid_box)
p.add_layout(low_box)
p.add_layout(top_box)

#### Point initial, final et d'arrivée

In [ ]:
# point initial
A = p.circle([0], [0], size=10, fill_color='#e32020', line_color='#e32020',)

# point final
B = p.circle([0], [l], size=10, fill_color='#e32020', line_color='#e32020')

# point d'arrivé
p_arrive = p.circle([d], [l], size=10, fill_color='#0A0451', line_color='#0A0451')

#### Vecteurs de différentes vitesses

In [ ]:
# vecteur vitesse par raport au courant
vit_ramer= p.line(v_ramer_x, v_ramer_y, color='#D94D31', line_width=1.5)


# vecteur vitesse courant
vit_courant = p.line(v_courant_x, v_courant_y, color='#130f78', line_width=1.5)


# vecteur vitesse par raport à la rive
vit_rive = p.line(np.linspace(0 ,v_courant_x[-1], 10), np.linspace(0, v_courant_y[-1], 10), color='#40ba2d',\
                   line_width=1.5)

#### Fonctions pour dessiner des flèches pour les vecteurs

In [ ]:
# fonctions pour dessiner des flèches 

# angle omega
omega= np.pi*20/180 # Cet angle sert à dessiner les têtes des flèches

def fleches_ramer(v_ramer_x, v_ramer_y, beta):
    haut_x = v_ramer_x[-1] + np.linspace(0, -100*np.cos(beta + omega), 10)
    haut_y = v_ramer_y[-1] + np.linspace(0, -100*np.sin(beta + omega), 10)
    
    bas_x = v_ramer_x[-1] + np.linspace(0, -100*np.cos(beta - omega), 10)
    bas_y = v_ramer_y[-1] + np.linspace(0, -100*np.sin(beta - omega), 10)
    
    return haut_x, haut_y, bas_x, bas_y


def fleches_courant(v_courant_x, v_courant_y):
    haut_x = v_courant_x[-1] + np.linspace(0, +100*np.cos(np.pi + omega), 10)
    haut_y = v_courant_y[-1] + np.linspace(0, +100*np.sin(np.pi + omega), 10)
    
    bas_x = v_courant_x[-1] + np.linspace(0, +100*np.cos(np.pi - omega), 10)
    bas_y = v_courant_y[-1] + np.linspace(0, +100*np.sin(np.pi - omega), 10)
    
    return haut_x, haut_y, bas_x, bas_y


def fleches_rive(v_courant_x, v_courant_y, alpha):
    haut_x = v_courant_x[-1] - np.linspace(0, 100*np.cos(alpha + omega), 10)
    haut_y = v_courant_y[-1] - np.linspace(0, 100*np.sin(alpha + omega), 10)
    
    bas_x = v_courant_x[-1] - np.linspace(0, 100*np.cos(alpha - omega), 10)
    bas_y = v_courant_y[-1] - np.linspace(0, 100*np.sin(alpha - omega), 10)
    
    return haut_x, haut_y, bas_x, bas_y


#### Flèches des vecteurs 

In [ ]:
# flèches vecteur vitesse ramer
r_haut_x, r_haut_y, r_bas_x, r_bas_y = fleches_ramer(v_ramer_x, v_ramer_y, beta)

fleche_haut_ramer = p.line(r_haut_x, r_haut_y, color='#D94D31', line_width=1.5)
fleche_bas_ramer = p.line(r_bas_x, r_bas_y, color='#D94D31', line_width=1.5)


# flèches vecteur vitesse courant
c_haut_x, c_haut_y, c_bas_x, c_bas_y = fleches_courant(v_courant_x, v_courant_y)

fleche_haut_courant = p.line(c_haut_x, c_haut_y, color='#130f78', line_width=1.5)
fleche_bas_courant = p.line(c_bas_x, c_bas_y, color='#130f78', line_width=1.5)

# flèches vecteur vitesse par raport à la rive
riv_haut_x, riv_haut_y, riv_bas_x, riv_bas_y = fleches_rive(v_courant_x, v_courant_y, alpha)

fleche_haut_v_rive = p.line(riv_haut_x, riv_haut_y, color='#40ba2d', line_width=1.5)
fleche_bas_v_rive = p.line(riv_bas_x, riv_bas_y, color='#40ba2d', line_width=1.5)

#### Trajectoires

In [ ]:
# trajectoire dans l'eau
trajectoire_eau = p.line(np.linspace(0, d*100, 10), np.linspace(0, l, 10),\
              color='#ccce2b', line_width=6, alpha=0.5, line_dash='dashed')

# trajectoire course
trajectoire_course = p.line(np.linspace(0, d*100, 10), np.linspace(l, l, 10),\
                           color='#2f4fec', line_width=6, alpha=0.9, line_dash='dotdash')

In [ ]:
# légende
legend1 = Legend(items=[("Vitesse par rapport au courant", [vit_ramer, fleche_haut_ramer, fleche_bas_ramer]),
                      ("Vitesse du courant", [vit_courant, fleche_haut_courant, fleche_bas_courant]),
                      ("Vitesse par rapport à la rive", [vit_rive, fleche_haut_v_rive, fleche_bas_v_rive])])
legend2 = Legend(items=[('Trajectoire Eau', [trajectoire_eau]),('Trajectoire Course', [trajectoire_course]), ('Point Initial', [A]), 
                       ('Point Final', [B]), ("Point d'Arrivé", [p_arrive])])

p.add_layout(legend1, 'below')
p.add_layout(legend2, 'below')
p.legend.orientation = 'horizontal'

### Temps

In [ ]:
p_temps = figure(title="Temps", plot_height=200, plot_width=980, y_range=(-10, 10), x_range=(0,3000), 
           background_fill_color='#ffffff', toolbar_location=None)

p_temps.yaxis.visible = None
p_temps.xaxis.axis_label = 'Temps [s]'

In [ ]:
# calculer temps 
temps_ramer = p_temps.line(np.linspace(0, t_r, 10), [0]*10,color='#6DD3DB', line_width=25, name='Temps ramer')
temps_course = p_temps.line(np.linspace(t_r, t_total, 10), [0]*10,color='#4a8d5c', line_width=25, name='Temps course')

In [ ]:
# légend
l_temps = Legend(items=[('Temps navigation [s]', [temps_ramer]), ('Temps courant [s]', [temps_course])])

p_temps.add_layout(l_temps)
p.ygrid.visible = False
p_temps.legend.orientation = 'horizontal'

### Fonction pour mettre à jour la figure

In [ ]:

# mettre à jour 
def update(beta):
    beta = np.pi*beta/180
    
    # mettre à jour vitesse
    v_ramer_x, v_ramer_y = v_ramer_(beta, v_ramer)
    v_courant_x = v_ramer_x[-1] + np.linspace(0, 200*v_courant, 10)
    v_courant_y = [v_ramer_y[-1]]*10
    alpha = np.arctan2(v_courant_y[-1],v_courant_x[-1])

    
    # mettre à jour temps
    t = t_course(v_courant, v_ramer, beta) + t_ramer(v_ramer, beta)

    point_arrive = (v_courant + v_ramer*np.cos(beta))*t_ramer(v_ramer,beta)
    
    temps_ramer.data_source.data['x'] = np.linspace(0,t_ramer(v_ramer,beta), 10)
    temps_course.data_source.data['x'] = np.linspace(t_ramer(v_ramer,beta), t, 10)
    
    # vecteur vitesse ramer
    vit_ramer.data_source.data['x'] = v_ramer_x
    vit_ramer.data_source.data['y'] = v_ramer_y
    
    # flèches du vecteur vitesse ramer
    r_haut_x, r_haut_y, r_bas_x, r_bas_y = fleches_ramer(v_ramer_x, v_ramer_y, beta)
    
    fleche_haut_ramer.data_source.data['x'] = r_haut_x
    fleche_haut_ramer.data_source.data['y'] = r_haut_y
    fleche_bas_ramer.data_source.data['x'] = r_bas_x
    fleche_bas_ramer.data_source.data['y'] = r_bas_y
    
    # vecteur vitesse courant
    vit_courant.data_source.data['x'] = v_courant_x
    vit_courant.data_source.data['y'] = v_courant_y
    
    # flèches de vectuer vitesse courant 
    c_haut_x, c_haut_y, c_bas_x, c_bas_y = fleches_courant(v_courant_x, v_courant_y)
    
    fleche_haut_courant.data_source.data['x'] = c_haut_x
    fleche_haut_courant.data_source.data['y'] = c_haut_y
    fleche_bas_courant.data_source.data['x'] = c_bas_x
    fleche_bas_courant.data_source.data['y'] = c_bas_y
    
    # vecteur vitesse rive
    vit_rive.data_source.data['x'] = np.linspace(0, v_courant_x[-1], 10)
    vit_rive.data_source.data['y'] = np.linspace(0, v_courant_y[-1], 10)
    
    riv_haut_x, riv_haut_y, riv_bas_x, riv_bas_y = fleches_rive(v_courant_x, v_courant_y, alpha)
    
    fleche_haut_v_rive.data_source.data['x'] =  riv_haut_x
    fleche_haut_v_rive.data_source.data['y'] = riv_haut_y
    fleche_bas_v_rive.data_source.data['x'] = riv_bas_x
    fleche_bas_v_rive.data_source.data['y'] = riv_bas_y
    
    p_arrive.data_source.data['x'] = [point_arrive]
    
    trajectoire_eau.data_source.data['x'] = np.linspace(0, point_arrive, 10)
    trajectoire_course.data_source.data['x'] = np.linspace(0, point_arrive, 10)
    
    print('Temps de navigation:\t\t{:0.1f} [s]'.format(t_ramer(v_ramer,beta)))
    print('Temps de course sur la rive:\t{:0.1f} [s]'.format(abs(t_course(v_courant, v_ramer, beta))))
    print('Temps total:\t\t\t{:0.1f} [s]'.format(t))
    
    push_notebook()
   



### Tracé

In [ ]:
f_plot = gridplot([[p], [p_temps]])
show(f_plot, notebook_handle=True)
interact(update,\
         beta=widgets.FloatSlider(min=70,max=160,step=1,value=90, description='$Beta$:')); 

### Analyse du temps en fonction de l'angle beta

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
v_ramer = 1.5 # vitesse à la rame (m/s)
v_course = 3.0 # vitesse à la course (m/s)

angle_beta_grad = np.linspace(60, 160, 99)

temps_ramer_beta = t_ramer(v_ramer, angle_beta_grad*np.pi/180)
temps_course_beta = abs(t_course(v_courant, v_ramer, angle_beta_grad*np.pi/180) )

temps_total_beta = temps_ramer_beta + temps_course_beta

t_ramer_idx = np.argmin(temps_ramer_beta)
t_course_idx = np.argmin(temps_course_beta)
t_total_idx = np.argmin(temps_total_beta)

# Graphique temps vs. angle beta
plt.figure(figsize=(14,7))
plt.plot(angle_beta_grad, temps_ramer_beta, color='green', linestyle='dashed', linewidth=2)
plt.plot(angle_beta_grad, temps_course_beta, color='blue', linestyle='dashed', linewidth=2)
plt.plot(angle_beta_grad, temps_total_beta, color='red', linewidth=2)

plt.plot(angle_beta_grad[t_ramer_idx], temps_ramer_beta[t_ramer_idx], color='green', marker='o', ms=8)
plt.plot(angle_beta_grad[t_course_idx], temps_course_beta[t_course_idx], color='blue', marker='o', ms=8)
plt.plot(angle_beta_grad[t_total_idx], temps_total_beta[t_total_idx], color='red', marker='o', ms=8)


plt.legend(['Temps traversée à la rame', 'Temps de course', 'Temps total'])
plt.grid(True)
plt.xlabel('Angle beta [º]')
plt.ylabel('Temps [s]')
plt.title('Temps vs Beta.\n (Vitesse Ramer = 1.5[m/s]. Vitesse Course = 3.0 [m/s])')
plt.show()

print("Beta pour le temps minimum à la rame {:0.1f}[º]. \n\
Temps ramer: {:0.1f}[s].\tTemps course: {:0.1f}[s].\tTemps total: {:0.1f}[s]\n".format(angle_beta_grad[t_ramer_idx], 
temps_ramer_beta[t_ramer_idx], temps_course_beta[t_ramer_idx], temps_total_beta[t_ramer_idx]))

print("Beta pour le temps minimum de course {:0.1f}[º]. \n\
Temps ramer: {:0.1f}[s].\tTemps course: {:0.1f}[s].\tTemps total: {:0.1f}[s]\n".format(angle_beta_grad[t_course_idx], 
temps_ramer_beta[t_course_idx], temps_course_beta[t_course_idx], temps_total_beta[t_course_idx]))

print("Beta pour le temps minimum total {:0.1f}[º]. \n\
Temps ramer: {:0.1f}[s].\tTemps course: {:0.1f}[s].\tTemps total: {:0.1f}[s]\n".format(angle_beta_grad[t_total_idx], 
temps_ramer_beta[t_total_idx], temps_course_beta[t_total_idx], temps_total_beta[t_total_idx]))

### Calcul analytique 2

Le temps total est une fonction de $\beta$:

$$\boxed{t_{tot}=\frac l{v'\sin\beta}+l\frac{v_c+v'\cos\beta}{v'\sin\beta}\frac1{v_{course}}=l\left[\frac{v_{course}+v_c+v'\cos\beta}{v_{course}\cdot v'\sin\beta}\right]}$$

On cherche à la minimiser en fonction de $\beta$:

$$\frac{\text dt_{tot}}{\text d\beta}=l\cdot\frac{-v_{course}v'\sin\beta\cdot v'\sin\beta-v_{course}v'\cos\beta\left[v_{course}+v_c+v'\cos\beta\right]}{\left[v_{course}\cdot v'\sin\beta\right]^2}=f(\beta)$$

et on cherche $f(\beta)=0$.

\begin{align*}
-v'^2v_{course}\sin^2\beta-v'v_{course}^2\cos\beta
-v_cv'v_{course}\cos\beta-v_{course}v'^2\cos^2\beta&=0\\
\end{align*}

$$\boxed{\cos\beta=\frac{-v'}{v_c+v_{course}}}$$
A.N.: $\beta=1.98$ (équivalent à 113.2°,  soit 23.2° vers l'amont). Le temps de course sera alors de 80.5 s et la traversée de 1160 s, soit un temps total de 1241.5 s (20.7 min).

# Demo: Variation des vitesses 

In [ ]:
def update2(beta, v_course, v_ramer, v_courant):
    beta = np.pi*beta/180
    
    # mettre à jour vitesse
    v_ramer_x = np.linspace(0, 200*np.cos(beta)*v_ramer, 10)
    v_ramer_y = np.linspace(0, 200*np.sin(beta)*v_ramer, 10)
    v_courant_x = v_ramer_x[-1] + np.linspace(0, 200*v_courant, 10)
    v_courant_y = [v_ramer_y[-1]]*10
    
    alpha = np.arctan2(v_courant_y[-1],v_courant_x[-1])

    
    # mettre à jour temps
    t = l*(v_course + v_courant + v_ramer*np.cos(beta))/(v_ramer*np.sin(beta) * v_course)
    t_r = t_ramer(v_ramer, beta)
    t_c = t - t_r

    point_arrive = (v_courant + v_ramer*np.cos(beta))*t_r
    
    temps_ramer.data_source.data['x'] = np.linspace(0,t_r, 10)
    temps_course.data_source.data['x'] = np.linspace(t_r, t, 10)
    
    # vecteur vitesse ramer
    vit_ramer.data_source.data['x'] = v_ramer_x
    vit_ramer.data_source.data['y'] = v_ramer_y
    
    # flèches du vecteur vitesse ramer
    r_haut_x, r_haut_y, r_bas_x, r_bas_y = fleches_ramer(v_ramer_x, v_ramer_y, beta)
    
    fleche_haut_ramer.data_source.data['x'] = r_haut_x
    fleche_haut_ramer.data_source.data['y'] = r_haut_y
    fleche_bas_ramer.data_source.data['x'] = r_bas_x
    fleche_bas_ramer.data_source.data['y'] = r_bas_y
    
    # vecteur vitesse courant
    vit_courant.data_source.data['x'] = v_courant_x
    vit_courant.data_source.data['y'] = v_courant_y
    
    # flèches de vectuer vitesse courant 
    c_haut_x, c_haut_y, c_bas_x, c_bas_y = fleches_courant(v_courant_x, v_courant_y)
    
    fleche_haut_courant.data_source.data['x'] = c_haut_x
    fleche_haut_courant.data_source.data['y'] = c_haut_y
    fleche_bas_courant.data_source.data['x'] = c_bas_x
    fleche_bas_courant.data_source.data['y'] = c_bas_y
    
    # vecteur vitesse rive
    vit_rive.data_source.data['x'] = np.linspace(0, v_courant_x[-1], 10)
    vit_rive.data_source.data['y'] = np.linspace(0, v_courant_y[-1], 10)
    
    riv_haut_x, riv_haut_y, riv_bas_x, riv_bas_y = fleches_rive(v_courant_x, v_courant_y, alpha)
    
    fleche_haut_v_rive.data_source.data['x'] =  riv_haut_x
    fleche_haut_v_rive.data_source.data['y'] = riv_haut_y
    fleche_bas_v_rive.data_source.data['x'] = riv_bas_x
    fleche_bas_v_rive.data_source.data['y'] = riv_bas_y
    
    p_arrive.data_source.data['x'] = [point_arrive]
    
    trajectoire_eau.data_source.data['x'] = np.linspace(0, point_arrive, 10)
    trajectoire_course.data_source.data['x'] = np.linspace(0, point_arrive, 10)
    
    print('Temps de navigation:\t\t{:0.1f} [s]'.format(t_r))
    print('Temps de course sur la rive:\t{:0.1f} [s]'.format(abs(t_c)))
    print('Temps total:\t\t\t{:0.1f} [s]'.format(t))
    
    push_notebook()
    
    
    
f_plot = gridplot([[p], [p_temps]])
show(f_plot, notebook_handle=True)
interact(update2,\
         beta=widgets.FloatSlider(min=90,max=179,step=1,value=113.2, description='$Beta$:'), \
        v_course=widgets.FloatSlider(min=0.5,max=5.0,step=0.05,value=3.0, description='$v_{course} [m/s]$:'),\
        v_ramer=widgets.FloatSlider(min=0.5,max=5,step=0.05,value=1.5, description='$v_{rame} [m/s]$:'),\
        v_courant=widgets.FloatSlider(min=0.1,max=3,step=0.05,value=0.8, description='$v_{c} [m/s]$:') ); 
         

# Feedback

In [ ]:
from IPython.display import IFrame
IFrame('https://www.surveymonkey.com/r/NOTOSURVEY?notebook_set=MecaDRIL&notebook_id=01-Logan', 600, 800)